In [2]:
import pandas as pd
import numpy as np
import requests

In [3]:
dataPop = requests.get('https://geo.api.gouv.fr/departements/16/communes')
DFPOP = pd.DataFrame(dataPop.json()) [['nom','population']]
DFREI = pd.read_csv('/home/dune/Téléchargements/Hackathon/REI_2022.csv')

DFP2 = pd.read_csv('/home/dune/Téléchargements/Hackathon/Ressources_HackathonDGFiP_novembre2023/GF-2C/160_CHARENTE_ART-P2.csv'
,header = 1, delimiter=';')
DFP1 = pd.read_csv('/home/dune/Téléchargements/Hackathon/Ressources_HackathonDGFiP_novembre2023/GF-2C/160_CHARENTE_ART-P1.csv'
,header = 1, delimiter=';',encoding='latin-1')
DFC1 = pd.read_csv('/home/dune/Téléchargements/Hackathon/Ressources_HackathonDGFiP_novembre2023/GF-2C/160_CHARENTE_ART-C1.csv'
,header = 1, delimiter=';',encoding='latin-1')
DFA1 = pd.read_csv('/home/dune/Téléchargements/Hackathon/Ressources_HackathonDGFiP_novembre2023/GF-2C/160_CHARENTE_ART-A1.csv'
,header = 1, delimiter=',',encoding='latin-1')


DFA1.columns

/tmp/ipykernel_50453/1393039754.py:3: DtypeWarning: Columns (0,4,16,23,29,32,36,51,72,81,86,96,106,109,112,184,200,205,331,335,339,340,354,355,359,360,364,365,366,370,382,386,390,877,882,883,884,885,886,906) have mixed types. Specify dtype option on import or set low_memory=False.
  DFREI = pd.read_csv('/home/dune/Téléchargements/Hackathon/REI_2022.csv')
/tmp/ipykernel_50453/1393039754.py:5: DtypeWarning: Columns (0,1,2,4,6,9,10,11,12,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,80) have mixed types. Specify dtype option on import or set low_memory=False.
  DFP2 = pd.read_csv('/home/dune/Téléchargements/Hackathon/Ressources_HackathonDGFiP_novembre2023/GF-2C/160_CHARENTE_ART-P2.csv'
/tmp/ipykernel_50453/1393039754.py:7: DtypeWarning: Columns (0,1,2,4,6,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40

Index(['CSDEP', 'CSDIR', 'CTCN', 'CTCNIA', 'NCCO', 'TYCNGR', 'CTCNGR',
       'COARCL', 'CTADR0', 'NIAF', 'NIAFBT', 'CITPCN', 'LTCN', 'NRTP',
       'DAIMRE', 'DAIMMJ', 'LTCU', 'CITPFI', 'CICNFI', 'CIRGFI', 'CIDPFI',
       'TXCNT0', 'TXCNS0', 'TXCNA0', 'TXCH2', 'TXCNU0', 'TXCNZ2', 'TXCH24',
       'TXCH23', 'TXCH0', 'TXCH1', 'MOCHD', 'MNDPDF', 'MNCHDF', 'MNCHD',
       'MBTP01', 'MBTP02', 'MBTP03', 'MBTP04', 'MBTP05', 'MBTP06', 'MBTP07',
       'MBTP08', 'MBTP09', 'MBTP10', 'MBTP11', 'MBTP12', 'MBTP13', 'MBTP14',
       'MBTP15', 'MBTP16', 'MBTP17', 'MBTP18', 'MBTP19', 'MBTP20', 'MBTP21',
       'MBTP22', 'MBTP23', 'MBTP24', 'FILLER', 'CICNGP', 'FILLER.1', 'BX001A'],
      dtype='object')

In [4]:
filtered_columns = [col for col in DFREI.columns if 'CFE' in col]
filtered_columns.append('Libellé commune')
DFREI = DFREI[filtered_columns]

# P2 : Base brute, réduction, montant rce
DFP2f = pd.concat([DFP2[['MBBS0','MOETA2','MOETR2','CTCN','NCCO']].iloc[0:1],DFP2[['MBBS0','MOETA2','MOETR2','CTCN','NCCO']].iloc[8:]])

# P1 : Base après réduction, base exonérée cfe commune, première exonération, deuxième exonération, base taxable commune
DFP1f = pd.concat([DFP1[['MBBSR', 'MNTPXC', 'CNEX01', 'CNEX02','MOBS20','CTCN','NCCO']].iloc[0:1],DFP1[['MBBSR', 'MNTPXC', 'CNEX01', 'CNEX02','MOBS20','CTCN','NCCO']].iloc[8:]])

# C1 : Nature de l'activité
DFC1f = pd.concat([DFC1[['CNAC3','CTCN','NCCO']].iloc[0:1], DFC1[['CNAC3','CTCN','NCCO']].iloc[8:]])

# A1 : Taux CFE commune
DFA1f = DFA1[['TXCNU0','CTCN','LTCN']]

In [5]:
merged_df = pd.merge(DFP2f, DFP1f, on=['NCCO','CTCN'], how='inner')
merged_df = pd.merge(merged_df, DFC1f, on=['NCCO','CTCN'], how='inner')
merged_df = pd.merge(merged_df, DFA1f, on=['CTCN'], how='left')
merged_df = merged_df.dropna()

colonnes = ['MBBS0', 'MOETA2', 'MOETR2', 'CTCN', 'NCCO', 'MBBSR', 'MNTPXC','CNEX01', 'CNEX02', 'MOBS20', 'CNAC3', 'TXCNT0']

merged_df[['CTCN','LTCN','TXCNU0','NCCO','CNAC3','MBBS0','MBBSR', 'MNTPXC', 'MOETA2', 'MOETR2','CNEX01', 'CNEX02', 'MOBS20']]

merged_df

,MBBS0,MOETA2,MOETR2,CTCN,NCCO,MBBSR,MNTPXC,CNEX01,CNEX02,MOBS20,CNAC3,TXCNU0,LTCN
0,base brute cfe annee n (10 car.),montant reduction artisans 10 car.,montant rce 10 car.,CODE COMMUNE DE L'AFT,numero de compte fichier coll CFE,base brute reduite n,base exoneree CFE commune,CODE NATURE PREMIERE EXO,CODE NATURE DEUXIEME EXO,base taxable commune n 10 car.,CODE NATURE ACTIVITE. APE NACE (FRP),taux cfe unique applicable 8 car. ...,Libelle commune au 1/1
5115,0,0,0,101,9930,0,0,,,0,6190Z,2572000,CLAIX
5116,0,0,0,101,9931,0,0,,,0,99904,2572000,CLAIX
5117,1268,0,0,101,9932,1268,0,,,0,4399C,2572000,CLAIX
5118,279,0,0,101,9933,279,0,,,0,3513Z,2572000,CLAIX
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8180,0,0,0,287,21441,0,0,,,0,8299Z,2572000,ROULLET ST ESTEPHE
8181,0,0,0,287,21442,0,0,MIN,,0,7410Z,2572000,ROULLET ST ESTEPHE
8182,0,0,0,287,21443,0,0,,,0,7490B,2572000,ROULLET ST ESTEPHE
8183,0,0,0,287,21444,0,0,,,0,6820A,2572000,ROULLET ST ESTEPHE


In [6]:
DFPOP.sort_values('population')[350:]

,nom,population
51,Brie,4222
251,Roullet-Saint-Estèphe,4295
147,Jarnac,4447
21,Barbezieux-Saint-Hilaire,4751
68,Champniers,5155
146,L'Isle-d'Espagnac,5633
136,Gond-Pontouvre,6017
254,Ruelle-sur-Touvre,7364
304,Saint-Yrieix-sur-Charente,7499
100,La Couronne,7742


In [8]:
DFGond = merged_df[merged_df['CTCN'] == '154']
print(len(DFGond))

DFLisle = merged_df[merged_df['CTCN'] == '166']
print(len(DFLisle))

553
426


In [19]:
DFLisle

,MBBS0,MOETA2,MOETR2,CTCN,NCCO,MBBSR,MNTPXC,CNEX01,CNEX02,MOBS20,CNAC3,TXCNU0,LTCN
6773,0,0,0,166,15281,0,0,,,0,6110Z,2572000,L'ISLE D'ESPAGNAC
6774,0,0,0,166,15282,0,0,,,0,6190Z,2572000,L'ISLE D'ESPAGNAC
6775,0,0,0,166,15283,0,0,MIN,,0,8219Z,2572000,L'ISLE D'ESPAGNAC
6776,247,0,0,166,15284,247,0,,,0,4321A,2572000,L'ISLE D'ESPAGNAC
6777,0,0,0,166,15285,0,0,MIN,,0,4799A,2572000,L'ISLE D'ESPAGNAC
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7194,0,0,0,166,15702,0,0,,,0,6190Z,2572000,L'ISLE D'ESPAGNAC
7195,0,0,0,166,15703,0,0,,,0,6190Z,2572000,L'ISLE D'ESPAGNAC
7196,517,0,0,166,15704,517,0,,,0,7830Z,2572000,L'ISLE D'ESPAGNAC
7197,1534,0,0,166,15705,1534,0,,,0,6512Z,2572000,L'ISLE D'ESPAGNAC


In [11]:
DFGond

,MBBS0,MOETA2,MOETR2,CTCN,NCCO,MBBSR,MNTPXC,CNEX01,CNEX02,MOBS20,CNAC3,TXCNU0,LTCN
6220,0,0,0,154,14490,0,0,,,0,6110Z,2572000,LE GOND PONTOUVRE
6221,0,0,0,154,14491,0,0,,,0,6190Z,2572000,LE GOND PONTOUVRE
6222,4129,0,0,154,14492,4129,0,,,0,2511Z,2572000,LE GOND PONTOUVRE
6223,0,0,0,154,14493,0,0,MIN,,0,4764Z,2572000,LE GOND PONTOUVRE
6224,1824,0,912,154,14494,912,0,,,0,4120A,2572000,LE GOND PONTOUVRE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6768,0,0,0,154,15038,0,0,MIN,,0,4782Z,2572000,LE GOND PONTOUVRE
6769,0,0,0,154,15039,0,0,MIN,,0,4331Z,2572000,LE GOND PONTOUVRE
6770,0,0,0,154,15040,0,0,,,0,6190Z,2572000,LE GOND PONTOUVRE
6771,0,0,0,154,15041,0,0,,,0,6190Z,2572000,LE GOND PONTOUVRE


In [16]:
unique_values = DFGond['CNEX01'].unique()

# Afficher les valeurs uniques
print("Valeurs uniques dans la colonne 'Category':")
print(unique_values)

unique_values = DFLisle['CNEX01'].unique()

# Afficher les valeurs uniques
print("Valeurs uniques dans la colonne 'Category':")
print(unique_values)

Valeurs uniques dans la colonne 'Category':
['   ' 'MIN' 'SPE' 'NVL' 'DPS']
Valeurs uniques dans la colonne 'Category':
['   ' 'MIN' 'NVL']


In [29]:
DF = DFLisle[DFLisle['CNEX01'] == 'MIN']['CNAC3']
print(DF)

DF = DFGond[DFGond['CNEX01'] == 'SPE']['CNAC3']
print(DF)

6775    8219Z
6777    4799A
6781    66221
6787    6820A
6793    9529Z
        ...  
7171    4110D
7179    4799A
7181    4932Z
7182    9329Z
7187    8690A
Name: CNAC3, Length: 107, dtype: object
6236    7010Z
Name: CNAC3, dtype: object


In [96]:
def depart(b1,b2,t1,t2, n1, n2,seuil=0.05, seuil2=0.15, seuil_concu = 0.5):
    #renvoie 1 si l'entreprise part de la ville 1 vers la ville 2 (ou reste dans la ville 2)

    C1 = b1*t1
    C2 = b2*t2

    conc = n2/(n1 + n2)

    if C1>(seuil +1)*C2 and conc < seuil_concu:
        return 1
    elif C1>(seuil2 + 1)*C2:
        return 1
    return 0

def pres_ent_ville(type,DFVille):
    return int(len(DFVille[DFVille['CNAC3']==type])!=0)

def fonction_objectif(types,DFVille1,DFVille2):
    f = 0
    for type in types : 
        for DFVille in  [DFVille1,DFVille2]:
            f+=pres_ent_ville(type,DFVille)
    return f


In [115]:
## Determination du nouveau mappage des entreprises sur les 2 villes, après modification des taux

def repartition_ent(Tf1,Tf2):
    
    L1 = []
    L2 = []

    for ent in Tf1['NCCO'].unique():
        type_ent = Tf1[Tf1['NCCO']==ent]['CNAC3'].unique()[0]
        L1.append([ent,type_ent])
    for ent in Tf2['NCCO'].unique():
        type_ent = Tf2[Tf2['NCCO']==ent]['CNAC3'].unique()[0]
        L2.append([ent,type_ent])
    return L1,L2

L1,L2 = repartition_ent(DFGond,DFLisle)
#print(L1)

def nb_ent_type(Rep,type_ent):
    n=0
    for ent in Rep:
        if ent[1]==type_ent:
            n+=1
    return n

#print(nb_ent_type(L2,'6420Z'))

In [116]:
def departs_successifs(type_ent,Li1,Li2,b1,b2,t1,t2, seuil=0.05, seuil2=0.15, seuil_concu = 0.5):
    n1 = nb_ent_type(Li1,type_ent)
    n2 = nb_ent_type(Li2,type_ent)
    
    Lf1 = []
    Lf2 = []


    for ent in Li1 :
        if ent[1]==type_ent:
            if depart(b1,b2,t1,t2, n1, n2,seuil, seuil2, seuil_concu):
                Lf2.append(ent)
                n2+=1
                n1-=1
            else:
                Lf1.append(ent)
    for ent in Li2 :
        if ent[1]==type_ent:
            if depart(b2,b1,t2,t1, n2, n1,seuil, seuil2, seuil_concu):
                Lf1.append(ent)
                n1+=1
                n2-=1
            else:
                Lf2.append(ent)  
    return Lf1,Lf2

departs_successifs('9329Z',L1,L2,1000,900,20,20)
    

([['14502', '9329Z'],
  ['14517', '9329Z'],
  ['14525', '9329Z'],
  ['14594', '9329Z'],
  ['14905', '9329Z']],
 [['14495', '9329Z'],
  ['14498', '9329Z'],
  ['15413', '9329Z'],
  ['15424', '9329Z'],
  ['15690', '9329Z']])

In [109]:
print(DFGond[DFGond['MBBSR']!='0'])
print(DFLisle[DFLisle['MBBSR']!='0'])

      MBBS0 MOETA2 MOETR2 CTCN   NCCO  MBBSR MNTPXC CNEX01 CNEX02 MOBS20  \
6222   4129      0      0  154  14492   4129      0                    0   
6224   1824      0    912  154  14494    912      0                    0   
6225  29500      0      0  154  14495  29500      0                    0   
6227   4036      0      0  154  14497   4036      0                    0   
6228   5646      0      0  154  14498   5646      0                    0   
...     ...    ...    ...  ...    ...    ...    ...    ...    ...    ...   
6762   1399      0      0  154  15032   1399      0                    0   
6764   3996      0      0  154  15034   3996      0                    0   
6765   7411      0      0  154  15035   7411      0                    0   
6766   2214      0      0  154  15036   2214      0                    0   
6767    205      0      0  154  15037    205      0                    0   

      CNAC3   TXCNU0                            LTCN  
6222  2511Z  2572000  LE GOND PO

In [206]:
def bases_type_ville(type_ent,DFVille1,DFVille2):
    b1 = DFVille1[DFVille1['CNAC3']==type_ent]['MBBSR'].unique()[0]
    b2 = DFVille1[DFVille1['CNAC3']==type_ent]['MBBSR'].unique()[0]


def bases_optimales(type_ent,b1,t1,b2,t2,Li1,Li2,prop = 0.5,eps = 0.15):

    #On décide de ne jouer que sur la commune 1

    iter = 0
    b1opt = b1
    b2opt = b2

    L1,L2 = departs_successifs(type_ent,Li1,Li2,b1opt,b2opt,t1,t2)
    n1 = nb_ent_type(L1,type_ent)
    n2 = nb_ent_type(L2,type_ent)

    while (n1/(n1+n2)<prop-eps or n1/(n1+n2)>prop+eps) and iter<1000:
        L1,L2 = departs_successifs(type_ent,L1,L2,b1opt,b2opt,t1,t2)

        n1 = nb_ent_type(L1,type_ent)
        n2 = nb_ent_type(L2,type_ent)
        iter+=1

        if b1opt*t1 < b2opt*t2:
            b1opt+=100
        else :
            b2opt+=100

    return b1opt,b2opt




In [207]:
Liste_Type_Activités1 = pd.concat([DFLisle,DFGond])['CNAC3'].unique()
len(Liste_Type_Activités1)
L = []

for type_ent in Liste_Type_Activités1:
    L.append([type_ent, nb_ent_type(L1,type_ent)+nb_ent_type(L2,type_ent)])
    #print(str(type_ent)+ " : " + str(nb_ent_type(L1,type_ent)+nb_ent_type(L2,type_ent)))

#LDF = pd.DataFrame(L)
#LDF = LDF.sort_values(1)
#LDF[255:]

Liste = []

for l in LDF[0][255:]:
    Liste.append(l)

#Liste

In [208]:
def bases_initiales(Types_Activites, DFVille1, DFVille2):

    bases_init = []
    L1,L2 = repartition_ent(DFGond,DFLisle)

    for type_ent in Types_Activites:
        n1 = nb_ent_type(L1,type_ent)
        n2 = nb_ent_type(L2,type_ent)

        if n1 == 0:
            b1 = 0
        else:
            b1 = int(DFVille1[DFVille1['CNAC3']==type_ent]['MBBSR'].unique()[0])
        if n2 == 0:
            b2 = 0
        else:
            b2 = int(DFVille2[DFVille2['CNAC3']==type_ent]['MBBSR'].unique()[0])
        bases_init.append([b1,b2])

    return bases_init

bases_initiales(Liste,DFGond,DFLisle)

[[7171, 2487],
 [0, 0],
 [8503, 138],
 [0, 0],
 [4036, 247],
 [0, 0],
 [0, 0],
 [2347, 0],
 [612, 0],
 [0, 0],
 [1571, 2503],
 [0, 0],
 [0, 0],
 [6719, 0],
 [0, 0]]

In [210]:
def bases_optimales_toutes_activites(Type_Activites,DFVille1,DFVille2,t1,t2,prop = 0.5,eps = 0.15):
    L = []
    base_init = bases_initiales(Type_Activites,DFVille1,DFVille2)
    L1,L2 = repartition_ent(DFVille1,DFVille2)

    for i in range(len(Type_Activites)-1):
        b1 = base_init[i][0]
        b2 = base_init[i][1]

        type_ent = Type_Activites[i]
        
        L.append(bases_optimales(type_ent,b1,t1,b2,t2,L1,L2,prop,eps))

    return L

bases_optimales_toutes_activites(Liste,DFGond,DFLisle,2752,2752)

[(54871, 54787),
 (50000, 50000),
 (54303, 54338),
 (0, 0),
 (52136, 52147),
 (50000, 50000),
 (0, 0),
 (51147, 51200),
 (712, 700),
 (0, 0),
 (52071, 52003),
 (50000, 50000),
 (0, 0),
 (53319, 53400)]